# **KM Master Discrepancy Detection System - Data Preprocessing**

![Python Code](https://img.shields.io/badge/Python-Code-blue?logo=python&logoColor=white)
![Sheets API](https://img.shields.io/badge/Google_Sheets-API-34A853?logo=googlesheets&logoColor=white)

### **Project Context**

KM Master represents the **round-trip distance** between Operating Points (OP) and Stores, which is critical for calculating **UJP** (Uang Jalan Pengiriman - transportation costs).

**Why This Matters:**
- **Underestimated distances** → Operational problems, driver dissatisfaction
- **Overestimated distances** → Fraud opportunities, inflated costs

With **thousands of stores**, manually validating all KM Master entries is impractical. The operations team needs to **prioritize** which entries to validate first.

# **Setup and Libraries**

In [1]:
# setup source path
import sys
import os

# add source directory to python path for custom modules
src_path = os.path.join(os.getcwd(), "..", "src")

if src_path not in sys.path:
    sys.path.append(src_path)
    print(f"Added {src_path[34:]} to sys.path")
else:
    print(f"{src_path[34:]} already in sys.path")

Added km-master-discrepancy-detection-system\notebooks\..\src to sys.path


In [2]:
# initialize configuration and environment variables
import config
from dotenv import load_dotenv
load_dotenv(config.ENV_PATH)

# set hide_values parameter
# config.HIDE_VALUES = False  # uncomment to show numeric values

# import custom modules
from data_preprocessing import convert_to_op_code, correct_scientific_notation
from utils import setup_logging, mask_numeric_value, DataTracker

# import common libraries
import pandas as pd
import numpy as np
import logging
import time
import re

# setup logging
setup_logging()
logger = logging.getLogger(__name__)

# display configuration
if config.HIDE_VALUES:
    logger.info(f"Hiding numeric values with '*' symbol")
else:
    logger.info("Showing numeric values")

2025-12-16 06:26:57 - __main__ - INFO - Hiding numeric values with '*' symbol


# **Data**

### **Data Preprocessing**

This notebook cleans and merges two data sources:

1. **Operational Data**
   - Historical delivery distances (KM Tempuh)
   
2. **Master Zona Data**
   - Store master information

**Output:**  
Clean, validated dataset ready for analysis

<br>

> **Disclaimer:** <br>Due to confidentiality, actual data **is not included** in this repository. <br>Some values are replaced with the `*` symbol.

## **A. Operational Data**

### **Key Columns:**
  - `Tgl SLA` - Delivery date
  - `OP` - Operating Point
  - `Nomor SPJ` - Delivery records / Waybill. One `SPJ` may contain several `Toko`
  - `Toko` - Store code
  - `KM Tempuh` - Actual distance traveled (round-trip) for each `SPJ`
  - `KM Master` - Master distance (round-trip) for each `Toko`
  - `KM Max` - Maximum value of `KM Master` for each `Toko` for each `SPJ`

### **1. Load Data & Start Tracker**

In [3]:
# define path for operational data
op_data_fns = ["1. Operational Data - Januari 2025.csv",
               "2. Operational Data - Februari 2025.csv",
               "3. Operational Data - Maret 2025.csv",
               "4. Operational Data - April 2025.csv",
               "5. Operational Data - Mei 2025.csv",
               "6. Operational Data - Juni 2025.csv",
               "7. Operational Data - Juli 2025.csv",
               "8. Operational Data - Agustus 2025.csv"]

op_data_paths = [os.path.join(config.DATA_PATH, "raw", filename) for filename in op_data_fns]

# read operational data
start_time = time.time()
df_op_data = pd.concat([pd.read_csv(path, sep=';', low_memory=False, dtype=str) for path in op_data_paths], ignore_index=True)
load_time = time.time() - start_time

logger.info(f"Loaded {mask_numeric_value(f'{len(df_op_data):,}')} rows of operational data in {load_time:.2f} secs")

2025-12-16 06:27:02 - __main__ - INFO - Loaded *,***,*** rows of operational data in 4.69 secs


In [4]:
# track preprocessing operational data
tracker_op_data = DataTracker("Operational Data")
tracker_op_data.track(df_op_data, step_name="Initial Rows")

2025-12-16 06:27:02 - utils - INFO - Initialized DataTracker for: [Operational Data]
2025-12-16 06:27:02 - utils - INFO - [Operational Data] Step: Initial Rows | Counts: *,***,*** | Change: +0 (+0.00%) | Retention: 100.00% | Step Time: 0.00s | Cumulative Time: 0.00s


### **2. Remove Empty or Duplicated Rows**

In [ ]:
# drop rows with all NaN value
df_op_data = df_op_data.dropna(how='all') 

tracker_op_data.track(df_op_data, step_name="Remove Empty Rows")

# drop rows with duplicate value
df_op_data = df_op_data.drop_duplicates()

tracker_op_data.track(df_op_data, step_name="Remove Duplicated Rows")

# validate if there are any null value
# it will help to identify data quality issues earlier than using dropna(how='any')
if df_op_data.isna().sum().sum() > 0:
    logger.exception("There are null values in the operational data")
    raise ValueError("There are null values in the operational data")

2025-12-16 06:27:04 - utils - INFO - [Operational Data] Step: Remove Empty Rows | Counts: *,***,*** | Change: -***,*** (-8.29%) | Retention: 91.71% | Step Time: 1.61s | Cumulative Time: 1.62s
2025-12-16 06:27:07 - utils - INFO - [Operational Data] Step: Remove Duplicated Rows | Counts: *,***,*** | Change: +0 (+0.00%) | Retention: 91.71% | Step Time: 2.91s | Cumulative Time: 4.52s


### **3. Convert Operating Point (OP) name to OP Code**

In [6]:
# convert OP name to OP code
# example: "Operating Point Satu" → "OP1"
# this function correct it using data from google sheets
df_op_data = convert_to_op_code(df_op_data, sheets_url=os.getenv('SHEETS_URL'))

tracker_op_data.track(df_op_data, step_name="Convert OP Name to OP Code")

2025-12-16 06:27:08 - data_preprocessing - INFO - Converting OP names to OP codes using method: complete
2025-12-16 06:27:08 - google_sheets_io - INFO - Loading Google Sheets from URL: https://docs.google.com/spread [Redacted]...
2025-12-16 06:27:08 - google_sheets_io - INFO - Credentials loaded successfully
2025-12-16 06:27:08 - google_sheets_io - INFO - Google Sheets API authorized
2025-12-16 06:27:09 - google_sheets_io - INFO - Sheet opened successfully
2025-12-16 06:27:10 - data_preprocessing - INFO - Loaded *** OP codes from sheets Master Kode OP
2025-12-16 06:27:16 - data_preprocessing - INFO - All OP names successfully converted to OP codes
2025-12-16 06:27:17 - utils - INFO - [Operational Data] Step: Convert OP Name to OP Code | Counts: *,***,*** | Change: +0 (+0.00%) | Retention: 91.71% | Step Time: 10.30s | Cumulative Time: 14.82s


### **4. Remove Ignored OPs and Stores**


In [7]:
# remove ignored OPs
# like testing OP, new OP, etc
exclude_ops = os.getenv('EXCLUDE_OPS').split(',')
df_op_data = df_op_data[~df_op_data['OP'].isin(exclude_ops)]

tracker_op_data.track(df_op_data, step_name="Remove Rows with Ignored OP")

# remove ignored stores
# like testing store, standby store, etc
exclude_stores = re.compile(rf"{os.getenv('EXCLUDE_STORES')}")
df_op_data = df_op_data[~df_op_data['Toko'].str.contains(exclude_stores, na=False)]

tracker_op_data.track(df_op_data, step_name="Remove Rows with Ignored Store")

2025-12-16 06:27:17 - utils - INFO - [Operational Data] Step: Remove Rows with Ignored OP | Counts: *,***,*** | Change: -**,*** (-0.30%) | Retention: 91.44% | Step Time: 0.51s | Cumulative Time: 15.34s
2025-12-16 06:27:19 - utils - INFO - [Operational Data] Step: Remove Rows with Ignored Store | Counts: *,***,*** | Change: -*,*** (-0.03%) | Retention: 91.41% | Step Time: 1.57s | Cumulative Time: 16.90s


### **5. Correct Scientific Notation**

In [8]:
# correct scientific notation
# excel sometimes convert store codes like "8E34" to "8.00E+34"
# this function correct it using data from google sheets
df_op_data = correct_scientific_notation(df_op_data, sheets_url=os.getenv('SHEETS_URL'))

tracker_op_data.track(df_op_data, step_name="Correct Scientific Notation")

2025-12-16 06:27:19 - data_preprocessing - INFO - Starting scientific notation correction for store codes
2025-12-16 06:27:19 - google_sheets_io - INFO - Loading Google Sheets from URL: https://docs.google.com/spread [Redacted]...
2025-12-16 06:27:19 - google_sheets_io - INFO - Credentials loaded successfully
2025-12-16 06:27:19 - google_sheets_io - INFO - Google Sheets API authorized
2025-12-16 06:27:20 - google_sheets_io - INFO - Sheet opened successfully
2025-12-16 06:27:20 - data_preprocessing - INFO - Loaded *** store mappings from sheets Master Saintifik Toko
2025-12-16 06:27:23 - data_preprocessing - INFO - Found *,*** stores with comma delimiter, * with period delimiter
2025-12-16 06:27:58 - data_preprocessing - INFO - All store codes successfully corrected.
2025-12-16 06:27:58 - utils - INFO - [Operational Data] Step: Correct Scientific Notation | Counts: *,***,*** | Change: +0 (+0.00%) | Retention: 91.41% | Step Time: 39.00s | Cumulative Time: 55.91s


### **6. Convert Data Types**

In [9]:
# convert data type
for col in ['KM Tempuh', 'KM Master', 'KM Max']:
  # remove '.' and ',' symbols
  df_op_data[col] = df_op_data[col].str.replace(r'[.,]', '', regex=True)
  # convert to numeric
  df_op_data[col] = pd.to_numeric(df_op_data[col], errors='coerce')
  # raise error if there is NaN data
  if df_op_data[col].isna().any():
    logger.error(f"Column {col} contains NaN values")
    raise ValueError(f"Column {col} contains NaN values")

tracker_op_data.track(df_op_data, step_name="Convert Data Type")

2025-12-16 06:28:08 - utils - INFO - [Operational Data] Step: Convert Data Type | Counts: *,***,*** | Change: +0 (+0.00%) | Retention: 91.41% | Step Time: 9.81s | Cumulative Time: 65.72s


### **7. Remove Rows with Invalid KM**

In [10]:
# remove rows with KM Tempuh < 1
df_op_data = df_op_data[df_op_data['KM Tempuh'] >= 1]

tracker_op_data.track(df_op_data, step_name="Remove Rows with Invalid KM")

2025-12-16 06:28:08 - utils - INFO - [Operational Data] Step: Remove Rows with Invalid KM | Counts: *,***,*** | Change: -*** (-0.02%) | Retention: 91.39% | Step Time: 0.33s | Cumulative Time: 66.05s


### **8. Calculate Deviation & Save Unique OP + Store**

In [11]:
# add column KM Deviation
df_op_data['KM Deviation (%)'] = (df_op_data['KM Tempuh'] - df_op_data['KM Max']) / df_op_data['KM Max'] * 100

# save tuple of unique OP and store for preprocessing master zona data
unique_pairs = df_op_data[['OP', 'Toko']].drop_duplicates()
unique_op_store_op_data = tuple(map(tuple, unique_pairs.values))

tracker_op_data.track(df_op_data, step_name="Calculate Deviation & Save Unique OP + Store")

2025-12-16 06:28:09 - utils - INFO - [Operational Data] Step: Calculate Deviation & Save Unique OP + Store | Counts: *,***,*** | Change: +0 (+0.00%) | Retention: 91.39% | Step Time: 0.79s | Cumulative Time: 66.83s


### **Summary**

In [12]:
# show summary
tracker_op_data.summary()

2025-12-16 06:28:09 - utils - INFO - [Operational Data] DataTracker summary generated for 10 steps.


,Step,Counts,Change,Change (%),Retained (%),Duration (s),Cumulative Time (s)
0,Initial Rows,"*,***,***",+0,+0.00,100.00,0.00,0.00
1,Remove Empty Rows,"*,***,***","-***,***",-8.29,91.71,1.61,1.62
2,Remove Duplicated Rows,"*,***,***",+0,+0.00,91.71,2.91,4.52
3,Convert OP Name to OP Code,"*,***,***",+0,+0.00,91.71,10.30,14.82
4,Remove Rows with Ignored OP,"*,***,***","-**,***",-0.30,91.44,0.51,15.34
5,Remove Rows with Ignored Store,"*,***,***","-*,***",-0.03,91.41,1.57,16.90
6,Correct Scientific Notation,"*,***,***",+0,+0.00,91.41,39.00,55.91
7,Convert Data Type,"*,***,***",+0,+0.00,91.41,9.81,65.72
8,Remove Rows with Invalid KM,"*,***,***",-***,-0.02,91.39,0.33,66.05
9,Calculate Deviation & Save Unique OP + Store,"*,***,***",+0,+0.00,91.39,0.79,66.83


## **B. Master Zona Data**

### **Key Columns:**
  - `OP` - Operating Point
  - `Toko` - Store code
  - `KM Master` - Master distance (round-trip) for each `Toko`
  - `Kode Zona` - Zone code
  - `Kecamatan`, `Kota`, `Provinsi` - Geographic info
  - `Status Toko` - Store active status (1=active, 0=inactive)

### **1. Load Data & Start Tracker**

In [13]:
# master zona data
mz_data_fn = "Master Zona Data.csv"
mz_data_path = os.path.join(config.DATA_PATH, "raw", mz_data_fn)

# read master zona data
start_time = time.time()
df_mz_data = pd.read_csv(mz_data_path, sep=';', low_memory=False, dtype=str, encoding='latin-1')
load_time = time.time() - start_time

logger.info(f"Loaded {mask_numeric_value(f'{len(df_mz_data):,}')} rows of master zona data in {load_time:.2f} secs")

2025-12-16 06:28:09 - __main__ - INFO - Loaded **,*** rows of master zona data in 0.14 secs


In [14]:
# track preprocessing master zona data
tracker_mz_data = DataTracker("Master Zona Data")
tracker_mz_data.track(df_mz_data, step_name="Initial Rows")

2025-12-16 06:28:09 - utils - INFO - Initialized DataTracker for: [Master Zona Data]
2025-12-16 06:28:09 - utils - INFO - [Master Zona Data] Step: Initial Rows | Counts: **,*** | Change: +0 (+0.00%) | Retention: 100.00% | Step Time: 0.00s | Cumulative Time: 0.00s


### **2. Remove Empty Rows, Rows Without Customer, or Duplicated Rows**

In [ ]:
# drop rows with all NaN value
df_mz_data = df_mz_data.dropna(how='all')

tracker_mz_data.track(df_mz_data, step_name="Remove Empty Rows")

# drop rows with NaN customer (no customer = no transaction)
df_mz_data = df_mz_data.dropna(subset=['Customer'])
tracker_mz_data.track(df_mz_data, step_name="Remove Rows Without Customer")

# drop rows with duplicate value
df_mz_data = df_mz_data.drop_duplicates()

tracker_mz_data.track(df_mz_data, step_name="Remove Duplicated Rows")

# validate if there are any null value
# it will help to identify data quality issues earlier than using dropna(how='any')
if df_mz_data.isna().sum().sum() > 0:
    logger.exception("There are null values in the master zona data")
    raise ValueError("There are null values in the master zona data")

2025-12-16 06:28:09 - utils - INFO - [Master Zona Data] Step: Remove Empty Rows | Counts: **,*** | Change: +0 (+0.00%) | Retention: 100.00% | Step Time: 0.06s | Cumulative Time: 0.06s
2025-12-16 06:28:09 - utils - INFO - [Master Zona Data] Step: Remove Rows Without Customer | Counts: **,*** | Change: -* (-0.01%) | Retention: 99.99% | Step Time: 0.02s | Cumulative Time: 0.08s
2025-12-16 06:28:09 - utils - INFO - [Master Zona Data] Step: Remove Duplicated Rows | Counts: **,*** | Change: +0 (+0.00%) | Retention: 99.99% | Step Time: 0.05s | Cumulative Time: 0.13s


### **3. Convert OP Name to OP Code using Partial Method**

In [16]:
# convert OP name to OP code with partial method
# use 'partial' method: drops rows with unmapped OPs instead of raising error
# reason: Master Zona may contain OPs outside our Operational data scope
df_mz_data = convert_to_op_code(df_mz_data, sheets_url=os.getenv("SHEETS_URL"), method='partial')

tracker_mz_data.track(df_mz_data, step_name="Convert OP Name to OP Code")

2025-12-16 06:28:09 - data_preprocessing - INFO - Converting OP names to OP codes using method: partial
2025-12-16 06:28:09 - google_sheets_io - INFO - Loading Google Sheets from URL: https://docs.google.com/spread [Redacted]...
2025-12-16 06:28:09 - google_sheets_io - INFO - Credentials loaded successfully
2025-12-16 06:28:09 - google_sheets_io - INFO - Google Sheets API authorized
2025-12-16 06:28:10 - google_sheets_io - INFO - Sheet opened successfully
2025-12-16 06:28:11 - data_preprocessing - INFO - Loaded *** OP codes from sheets Master Kode OP
2025-12-16 06:28:11 - data_preprocessing - INFO - Dropped *,*** rows with unmapped OP codes (10.31%%)
2025-12-16 06:28:11 - utils - INFO - [Master Zona Data] Step: Convert OP Name to OP Code | Counts: **,*** | Change: -*,*** (-10.31%) | Retention: 89.68% | Step Time: 1.85s | Cumulative Time: 1.98s


### **4. Correct Scientific Notation**

In [17]:
# correct scientific notation
df_mz_data = correct_scientific_notation(df_mz_data, sheets_url=os.getenv("SHEETS_URL"))

tracker_mz_data.track(df_mz_data, step_name="Correct Scientific Notation")

2025-12-16 06:28:11 - data_preprocessing - INFO - Starting scientific notation correction for store codes
2025-12-16 06:28:11 - google_sheets_io - INFO - Loading Google Sheets from URL: https://docs.google.com/spread [Redacted]...
2025-12-16 06:28:11 - google_sheets_io - INFO - Credentials loaded successfully
2025-12-16 06:28:11 - google_sheets_io - INFO - Google Sheets API authorized
2025-12-16 06:28:12 - google_sheets_io - INFO - Sheet opened successfully
2025-12-16 06:28:12 - data_preprocessing - INFO - Loaded *** store mappings from sheets Master Saintifik Toko
2025-12-16 06:28:13 - data_preprocessing - INFO - Found ** stores with comma delimiter, ** with period delimiter
2025-12-16 06:28:13 - data_preprocessing - INFO - All store codes successfully corrected.
2025-12-16 06:28:13 - utils - INFO - [Master Zona Data] Step: Correct Scientific Notation | Counts: **,*** | Change: +0 (+0.00%) | Retention: 89.68% | Step Time: 2.18s | Cumulative Time: 4.16s


### **5. Filter to Operational Scope**

In [18]:
# keep only stores that exist in operational data
df_mz_data = df_mz_data[df_mz_data[['OP', 'Toko']].apply(tuple, axis=1).isin(unique_op_store_op_data)]

tracker_mz_data.track(df_mz_data, step_name="Filter to Operational Scope")

2025-12-16 06:28:13 - utils - INFO - [Master Zona Data] Step: Filter to Operational Scope | Counts: **,*** | Change: -**,*** (-36.55%) | Retention: 56.90% | Step Time: 0.33s | Cumulative Time: 4.48s


### **6. Convert Data Types**

In [19]:
# convert data type
for col in ['KM Master']:
  # remove '.' and ',' symbols
  df_mz_data[col] = df_mz_data[col].str.replace(r'[.,]', '', regex=True)
  # convert to numeric
  df_mz_data[col] = pd.to_numeric(df_mz_data[col], errors='coerce')
  # raise error if there is NaN data
  if df_mz_data[col].isna().any():
    logger.error(f"Column {col} contains NaN values")
    raise ValueError(f"Column {col} contains NaN values")

tracker_mz_data.track(df_mz_data, step_name="Convert Data Type")

2025-12-16 06:28:14 - utils - INFO - [Master Zona Data] Step: Convert Data Type | Counts: **,*** | Change: +0 (+0.00%) | Retention: 56.90% | Step Time: 0.04s | Cumulative Time: 4.53s


### **Summary**

In [20]:
# show summary
tracker_mz_data.summary()

2025-12-16 06:28:14 - utils - INFO - [Master Zona Data] DataTracker summary generated for 8 steps.


,Step,Counts,Change,Change (%),Retained (%),Duration (s),Cumulative Time (s)
0,Initial Rows,"**,***",+0,+0.00,100.00,0.00,0.00
1,Remove Empty Rows,"**,***",+0,+0.00,100.00,0.06,0.06
2,Remove Rows Without Customer,"**,***",-*,-0.01,99.99,0.02,0.08
3,Remove Duplicated Rows,"**,***",+0,+0.00,99.99,0.05,0.13
4,Convert OP Name to OP Code,"**,***","-*,***",-10.31,89.68,1.85,1.98
5,Correct Scientific Notation,"**,***",+0,+0.00,89.68,2.18,4.16
6,Filter to Operational Scope,"**,***","-**,***",-36.55,56.90,0.33,4.48
7,Convert Data Type,"**,***",+0,+0.00,56.90,0.04,4.53


## **C. All Data**

### **Merge Strategy**
  - Using `inner join` because we only analyze OP and Stores in both Operational and Master Zona data
  - Join keys using `OP` and `Toko` because there is no one distinct column. It is because one `OP` may has several `Toko` and vice versa
  - Because both data has `KM Master` column, only `KM Master` column from Master Zona data will be used. It is because in Operational data, there is may outdated `KM Master` value in past

### **1. Merged Data & Start Tracker**

In [21]:
# merged data
start_time = time.time()
df_all_data = pd.merge(
                df_op_data[[col for col in df_op_data.columns if col != "KM Master"]],
                df_mz_data[
                    ['OP','Toko','KM Master','Kode Zona',
                     'Kecamatan','Kota','Provinsi','Status Toko']],
                how='inner', on=['OP', 'Toko']
                )
load_time = time.time() - start_time

logger.info(f"Combined {mask_numeric_value(f'{len(df_all_data):,}')} rows of operational data and master zona data in {load_time:.2f} secs")

2025-12-16 06:28:16 - __main__ - INFO - Combined *,***,*** rows of operational data and master zona data in 2.26 secs


In [22]:
# track preprocessing merged data
tracker_all_data = DataTracker("All Data")
tracker_all_data.track(df_all_data, step_name="Initial Rows")

2025-12-16 06:28:16 - utils - INFO - Initialized DataTracker for: [All Data]
2025-12-16 06:28:16 - utils - INFO - [All Data] Step: Initial Rows | Counts: *,***,*** | Change: +0 (+0.00%) | Retention: 100.00% | Step Time: 0.00s | Cumulative Time: 0.00s


### **2. Remove Empty or Duplicated Rows**

In [ ]:
# drop rows with all NaN value
df_all_data = df_all_data.dropna(how='all')

tracker_all_data.track(df_all_data, step_name="Remove Empty Rows")

# drop rows with duplicate value
df_all_data = df_all_data.drop_duplicates()

tracker_all_data.track(df_all_data, step_name="Remove Duplicated Rows")

# validate if there are any null value
# it will help to identify data quality issues earlier than using dropna(how='any')
if df_all_data.isna().sum().sum() > 0:
    logger.exception("There are null values in the all data")
    raise ValueError("There are null values in the all data")

2025-12-16 06:28:18 - utils - INFO - [All Data] Step: Remove Empty Rows | Counts: *,***,*** | Change: +0 (+0.00%) | Retention: 100.00% | Step Time: 2.20s | Cumulative Time: 2.20s
2025-12-16 06:28:23 - utils - INFO - [All Data] Step: Remove Duplicated Rows | Counts: *,***,*** | Change: -** (-0.00%) | Retention: 100.00% | Step Time: 5.28s | Cumulative Time: 7.48s


### **3. Remove Inactive Store**

In [24]:
# filter status toko = 1
df_all_data = df_all_data[df_all_data['Status Toko'] == '1']

# drop column Status Toko
df_all_data = df_all_data.drop(columns=['Status Toko'])

tracker_all_data.track(df_all_data, step_name="Remove Inactive Store")

2025-12-16 06:28:26 - utils - INFO - [All Data] Step: Remove Inactive Store | Counts: *,***,*** | Change: -*,*** (-0.13%) | Retention: 99.87% | Step Time: 2.61s | Cumulative Time: 10.09s


### **Summary**

In [25]:
# show summary
tracker_all_data.summary()

2025-12-16 06:28:26 - utils - INFO - [All Data] DataTracker summary generated for 4 steps.


,Step,Counts,Change,Change (%),Retained (%),Duration (s),Cumulative Time (s)
0,Initial Rows,"*,***,***",+0,+0.00,100.00,0.00,0.00
1,Remove Empty Rows,"*,***,***",+0,+0.00,100.00,2.20,2.20
2,Remove Duplicated Rows,"*,***,***",-**,-0.00,100.00,5.28,7.48
3,Remove Inactive Store,"*,***,***","-*,***",-0.13,99.87,2.61,10.09


# **Final Result**

## **Save All Data**

In [26]:
# save clean data
output_file = os.path.join(config.DATA_PATH, "clean", "df_all_data.csv")

start_time = time.time()
df_all_data.to_csv(output_file, index=False)
load_time = time.time() - start_time

file_size = os.path.getsize(output_file) / (1024**2) # in MB

logger.info(f"Saved {mask_numeric_value(f'{len(df_all_data):,}')} rows to {output_file[93:]} ({file_size:.2f} MB) in {load_time:.2f} secs")

2025-12-16 06:28:45 - __main__ - INFO - Saved *,***,*** rows to data\clean\df_all_data.csv (579.72 MB) in 19.19 secs


## **Important Note**

In [27]:
# show unique OP + store
print(f"All Data has {mask_numeric_value(f'{len(df_all_data):,}')} rows which is consist from {mask_numeric_value(f'{len(df_all_data[['OP', 'Toko']].drop_duplicates()):,}')} unique OP and Store combinations")

All Data has *,***,*** rows which is consist from **,*** unique OP and Store combinations


All Data contains:
- **Total Rows**: \*,\*\*\*,\*\*\* delivery records (SPJ)
- **Unique OP and Store Combination**: \*\*,\*\*\* stores

This is important because each store may have multiple delivery records over time.

Example:

- Store 'ABCD' at OP 'OP1' might have **50 delivery records**
- However, it still count as **1 unique store**

<br>

> For further analysis, this project will analyze at the **unique store** (unique OP and Store combination), not at **delivery records** (total rows) level